<a href="https://colab.research.google.com/github/Saranyanv/data-go/blob/master/Misc/sudoku_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import random
import copy
import pprint

pp = pprint.PrettyPrinter(indent=6)

def difference(list1):
   list_dif = [i for i in list1 + [1,2,3,4,5,6,7,8,9] if (i not in list1 or i not in [1,2,3,4,5,6,7,8,9]) and i!=0]
   return list_dif

def diff_list(list1, list2):
   list_dif = [i for i in list1 if (i not in list1 or i not in list2)]
   return list_dif

def uniq_append(element, list1):
  if element not in list1 and element!=0:
    list1.append(element)

def dupcheck(row_num, col_num, arg_list):
  tmp_dict={}
  tmp=[]
  for i in [i for i in arg_list[row_num] if i!=0 and type(i)==type(1)]:
    if i not in tmp_dict.keys():
      tmp_dict[i]=0
      uniq_append(i,tmp)
    #else:
      #return 'Dups'
  
  tmp_dict={}
  for n in range(9):
    if arg_list[n][col_num]==0:
      pass
    elif type(arg_list[n][col_num])==type(1) and arg_list[n][col_num] not in tmp_dict.keys():
      tmp_dict[arg_list[n][col_num]]=0
      uniq_append(arg_list[n][col_num],tmp)
    #else:
      #return 'Dups'
    
  tmp_dict={}
  for i1 in range(row_num-np.mod(row_num,3), row_num+np.mod(8-row_num,3)+1):
    for j1 in range(col_num-np.mod(col_num,3), col_num+np.mod(8-col_num,3)+1):
      if arg_list[i1][j1]==0:
        pass
      elif type(arg_list[i1][j1])==type(1) and arg_list[i1][j1] not in tmp_dict.keys():
        tmp_dict[arg_list[i1][j1]]=0
        uniq_append(arg_list[i1][j1],tmp)
      #else:
        #return 'Dups'

  return sorted(difference(tmp))
  #if difference(tmp)==[]:
    #return 'Dups'
  #else:
    #return sorted(difference(tmp))

In [0]:
rows, cols = (9, 9) 

In [0]:
def print_matrix(arg_list):
  try:
    offset=(max([len(i) for j in restart_list for i in j if type(i)==type([])])*3)+1
  except ValueError:
    offset=2
  for i in range(rows):
    tmp=''
    for j in range(cols):
      if type(j)==type([]):
        str_len=offset-len(j)
      else:
        str_len=offset-1
      tmp+=" "+str(arg_list[i][j]).ljust(str_len)+"|"
    print (tmp)

In [0]:
def dedupe_rows(row_num,arg_list):
  # Idenfity Naked Doubles and dedupe the corresponding row
  for r in range(row_num):
      dup={}
      for j in arg_list[r]:
        if type(j)==type([]):
          if tuple(j) not in dup.keys():
            dup[tuple(j)]=1
          else:
            dup[tuple(j)]=dup[tuple(j)]+1
        else:
          if j not in dup.keys():
            dup[j]=1
          else:
            dup[j]=dup[j]+1
      for (k,v) in dup.items():
        if type(k)==type(tuple()) and len(k)==v:
          for j in k:
            for i in range(9):
              if type(arg_list[r][i])==type([]) and arg_list[r][i]!=list(k) and j in arg_list[r][i]:
                arg_list[r][i].remove(j)

      # Idenfity Isolated Singles and dedupe the corresponding row
      dup={}
      for element in arg_list[r]:
        if type(element)==type([]):
          for idx,val in enumerate(element):
            if val not in dup.keys():
              dup[val]=1
            else:
              dup[val]=dup[val]+1
        elif type(element)==type(1):
          if element not in dup.keys():
            dup[element]=1
          else:
            dup[element]=dup[element]+1
      for j in [k for (k,v) in dup.items() if v==1]:
        for i in range(9):
          if type(arg_list[r][i])==type([]) and len(arg_list[r][i]) > 1 and j in arg_list[r][i]:
            arg_list[r][i]=j

  return len([i for j in arg_list for i in j if type(i)==type([]) and len(i)>1])

In [0]:
def dedupe_cols(col_num,arg_list):
    for i in range(col_num):
        dup={}
        for j in range(cols):
            if type(arg_list[j][i])==type([]):
                if tuple(arg_list[j][i]) not in dup.keys():
                    dup[tuple(arg_list[j][i])]=1
                else:
                    dup[tuple(arg_list[j][i])]=dup[tuple(arg_list[j][i])]+1
            else:
                if arg_list[j][i] not in dup.keys():
                    dup[arg_list[j][i]]=1
                else:
                    dup[arg_list[j][i]]=dup[arg_list[j][i]]+1
        for (k,v) in dup.items():
            if type(k)==type(tuple()) and len(k)==v:
                for j in k:
                    for r in range(9):
                        if type(arg_list[r][i])==type([]) and arg_list[r][i]!=list(k) and j in arg_list[r][i]:
                            arg_list[r][i].remove(j)

        for i in range(cols):
          dup={}
          for j in range(rows):
            if type(arg_list[j][i])==type([]): #and len(arg_list[j][i])>1:
              for idx,val in enumerate(arg_list[j][i]):
                if val not in dup.keys():
                  dup[val]=1
                else:
                  dup[val]=dup[val]+1
            else:
                if arg_list[j][i] not in dup.keys():
                  dup[arg_list[j][i]]=1
                else:
                  dup[arg_list[j][i]]=dup[arg_list[j][i]]+1              
          for j1 in [k for (k,v) in dup.items() if v==1]:
            for n in range(9):
              if type(arg_list[n][i])==type([]) and j1 in arg_list[n][i] and len(arg_list[n][i])>1:
                arg_list[n][i]=j1
    return len([i for j in arg_list for i in j if type(i)==type([]) and len(i)>1])

In [0]:
'''def dedupe_squares(arg_list):
  row_start=0
  for i in range(0,3):
    col_start=0
    for j in range(0,3):
      lst={}
      chk=0
      pos=[]
      dup={}
      for row in range(row_start,row_start+3):
        for col in range(col_start,col_start+3):
          if type(arg_list[row][col])==type([]):
            lst[tuple([row,col])]=arg_list[row][col]
            for idx,val in enumerate(arg_list[row][col]):
              if val not in dup.keys():
                dup[val]=1
              else:
                dup[val]=dup[val]+1
          else:
              if arg_list[row][col] not in dup.keys():
                dup[arg_list[row][col]]=1
              else:
                dup[arg_list[row][col]]=dup[arg_list[row][col]]+1  

      for row in range(row_start,row_start+3):
        for col in range(col_start,col_start+3):
          for j1 in [k for (k,v) in dup.items() if v==1]:
            if type(arg_list[row][col])==type([]) and j1 in arg_list[row][col] and len(arg_list[row][col])>1:
              arg_list[row][col]=j1

      tmp={}
      #print (tmp)
      for j in lst.values():
        for i in lst.values():
          if type(i)==type([]) and type(j)==type([]) and set(i).issubset(set(j))==True and i!=j:
            for element in i:
              if element in [j1 for i1 in lst.values() for j1 in i1 if i1!=j and i1!=i]:
                chk+=1
              else:
                chk+=0
            if chk==0:
              tmp[tuple(i)]=j
      for (k,v) in tmp.items():
        pos=[key for (key, value) in lst.items() if value == v]
        arg_list[pos[0][0]][pos[0][1]]=list(k)
        #print ('updated ',pos,'with',k,tmp)
      #print (row_start, col_start)
      col_start+=3
    row_start+=3
  return len([i for j in arg_list for i in j if type(i)==type([]) and len(i)>1])    
'''

def dedupe_squares(arg_list):
  row_start=0
  for i in range(0,3):
    col_start=0
    for j in range(0,3):
      lst={}
      chk=0
      pos=[]
      for row in range(row_start,row_start+3):
        for col in range(col_start,col_start+3):
          if type(arg_list[row][col])==type([]):
            lst[tuple([row,col])]=arg_list[row][col]
      tmp={}
      #print (tmp)
      for j in lst.values():
        for i in lst.values():
          if type(i)==type([]) and type(j)==type([]) and set(i).issubset(set(j))==True and i!=j:
            for element in i:
              if element in [j1 for i1 in lst.values() for j1 in i1 if i1!=j and i1!=i]:
                chk+=1
              else:
                chk+=0
            if chk==0:
              tmp[tuple(i)]=j
      for (k,v) in tmp.items():
        pos=[key for (key, value) in lst.items() if value == v]
        arg_list[pos[0][0]][pos[0][1]]=list(k)
        #print ('updated ',pos,'with',k,tmp)
      #print (row_start, col_start)
      col_start+=3
    row_start+=3
  return len([i for j in arg_list for i in j if type(i)==type([]) and len(i)>1])   

In [0]:
def prepare_output(arg_list):
    solved_sudoku=[]
    for row in arg_list:
        solved_sudoku_tmp=[]
        for element in row:
            if type(element)==type([]):
                solved_sudoku_tmp.append(int(element[0]))
            else:
                solved_sudoku_tmp.append(element)
        solved_sudoku.append(solved_sudoku_tmp)
    return solved_sudoku

In [119]:
### Complexity=Medium
#arr_sudoku=[[0,0,3,1,0,0,0,0,0],[0,5,6,9,0,0,0,0,1],[0,0,0,0,7,0,8,0,2],[0,0,0,0,0,0,0,4,3],[0,8,9,0,3,0,7,2,0],[3,6,0,0,0,0,0,0,0],[9,0,1,0,5,0,0,0,0],[5,0,0,0,0,7,4,3,0],[0,0,0,0,0,9,1,0,0]]

### Complexity=Hard
#arr_sudoku=[[5,0,0,0,0,0,1,6,2],[0,3,0,2,0,0,0,0,0],[0,0,6,1,0,0,8,0,0],[0,0,5,9,0,0,0,2,0],[0,0,0,4,5,2,0,0,0],[0,6,0,0,0,3,4,0,0],[0,0,2,0,0,1,7,0,0],[0,0,0,0,0,8,0,3,0],[9,1,7,0,0,0,0,0,5]]
arr_sudoku=[[7,0,0,0,4,0,1,0,0],[0,2,5,3,6,0,0,0,0],[6,0,0,0,0,0,5,8,0],[8,0,3,2,0,0,0,4,0],[0,0,0,0,0,0,0,0,0],[0,1,0,0,0,4,7,0,8],[0,6,7,0,0,0,0,0,4],[0,0,0,0,2,6,8,3,0],[0,0,9,0,7,0,0,0,1]]
'''arr_sudoku=[[0,0,0,9,0,7,0,0,0],
     [0,1,0,0,0,0,5,7,0],
     [0,0,0,8,2,0,0,0,3],
     [3,0,0,5,0,0,6,0,7],
     [7,0,0,0,0,0,0,0,4],
     [2,0,1,0,0,4,0,0,9],
     [9,0,0,0,4,6,0,0,0],
     [0,6,2,0,0,0,0,3,0],
     [0,0,0,3,0,5,0,0,0]]'''

restart_list=[[0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0],
     [0,0,0,0,0,0,0,0,0]]
     
not_filled=len([i for j in arr_sudoku for i in j if i==0])
prev_not_filled=0;
STUS='IPRO'

print ("Sudoku..")
print(np.matrix(arr_sudoku))
# Solve for Naked singles
while STUS!='COMP':
    for i in range(rows):
        for j in range(cols):
            if arr_sudoku[i][j]<=0:
                dup_chk=dupcheck(i,j,arr_sudoku)
                if len(dup_chk)==1:
                    arr_sudoku[i][j]=sum(int(x) for x in dup_chk)
                    restart_list[i][j]=arr_sudoku[i][j]
                else:
                    restart_list[i][j]=dup_chk
            else:
                restart_list[i][j]=arr_sudoku[i][j]
    prev_not_filled=not_filled
    not_filled=len([i for j in arr_sudoku for i in j if i==0])
    if not_filled==0:
      STUS='COMP'
    elif prev_not_filled==not_filled:
      #print_matrix(restart_list)
       #Solve for Hidden singles, doubles, triples
      while STUS=='IPRO':
        unfilled_row_count=dedupe_rows(rows,restart_list)
        #print ('after deduping rows')
        #print_matrix(restart_list)
        unfilled_col_count=dedupe_cols(cols,restart_list)
        #print ('after deduping columns')
        #print_matrix(restart_list)
        unfilled_square_count=dedupe_squares(restart_list)
        #print ('after deduping squares')
        #print_matrix(restart_list)
        for i in range(rows):
          for j in range(cols):
            if type(restart_list[i][j])==type([]) and len(restart_list[i][j]) >= 1:
              dup_chk=dupcheck(i,j,restart_list)
              if len(dup_chk)==1:
                restart_list[i][j]=sum(int(x) for x in dup_chk)
              elif len(restart_list[i][j]) >= len(dup_chk):
                restart_list[i][j]=dup_chk
        #print ('after deduping squares 2')
        #print_matrix(restart_list)
        total_unfilled=len([i for j in restart_list for i in j if type(i)==type([]) and len(i)>1])
        if total_unfilled==0:
        #if unfilled_row_count==0 and unfilled_col_count==0:
          output=prepare_output(restart_list)
          STUS='COMP'
          print(" ")
          print("Solved Sudoku..")
          print(np.matrix(output))
          break

Sudoku..
[[7 0 0 0 4 0 1 0 0]
 [0 2 5 3 6 0 0 0 0]
 [6 0 0 0 0 0 5 8 0]
 [8 0 3 2 0 0 0 4 0]
 [0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 4 7 0 8]
 [0 6 7 0 0 0 0 0 4]
 [0 0 0 0 2 6 8 3 0]
 [0 0 9 0 7 0 0 0 1]]
 
Solved Sudoku..
[[7 3 8 5 4 9 1 2 6]
 [1 2 5 3 6 8 4 7 9]
 [6 9 4 7 1 2 5 8 3]
 [8 7 3 2 9 1 6 4 5]
 [9 4 6 8 5 7 3 1 2]
 [5 1 2 6 3 4 7 9 8]
 [2 6 7 1 8 3 9 5 4]
 [4 5 1 9 2 6 8 3 7]
 [3 8 9 4 7 5 2 6 1]]


In [120]:
print_matrix(output)

 7| 3| 8| 5| 4| 9| 1| 2| 6|
 1| 2| 5| 3| 6| 8| 4| 7| 9|
 6| 9| 4| 7| 1| 2| 5| 8| 3|
 8| 7| 3| 2| 9| 1| 6| 4| 5|
 9| 4| 6| 8| 5| 7| 3| 1| 2|
 5| 1| 2| 6| 3| 4| 7| 9| 8|
 2| 6| 7| 1| 8| 3| 9| 5| 4|
 4| 5| 1| 9| 2| 6| 8| 3| 7|
 3| 8| 9| 4| 7| 5| 2| 6| 1|
